## Voronoi expansion implementation in DEVS Simulation Framework 


In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Attemping to make an animated matplotlib celldevs plot
Using as example project a Vornoi Zones implementation in cellDevs.

See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [71]:
import numpy as np
import matplotlib
import matplotlib.animation as animation

from matplotlib import pyplot as plt
from IPython.display import HTML
from pycdevs.cdppwrapper import CDPPWrapper, DrawlogFailedException
from pycdevs.models import Model, CellValues
from pycdevs import cdppwrapper as cdppExceptions
from pycdevs import voronoi

matplotlib.rcParams['figure.figsize'] = (10,8)

Snippet to generate a randomly initiated cell values, for a voronoi model of a certain size

In [64]:
import random

inital_cell_values = np.zeros((40, 40), dtype=int)
get_random_i = lambda : random.randint(0, inital_cell_values.shape[0]-1)
get_random_j = lambda : random.randint(0, inital_cell_values.shape[1]-1)
current_value = 1

for _ in range(5):
    random_i = get_random_i()
    random_j = get_random_j()
    inital_cell_values[random_i, random_j] = current_value
    print("Initiating ({0},{1}) with value {2}".format(random_i, random_j, current_value))
    current_value += 1
    
devs_initiall_cell_values = CellValues(inital_cell_values, 0)

Initiating (30,39) with value 1
Initiating (33,22) with value 2
Initiating (27,29) with value 3
Initiating (29,6) with value 4
Initiating (35,10) with value 5


In [4]:
# diamond shaped neighbourhood
diamond_neighbours = [
    (0, -2), \
    (-1,-1), (0, -1), (1,-1), \
    (-2, 0), (-1,0), (1,0), (2, 0),\
    (-1,1), (0, 1), (1,1), \
    (0, 2)
]

diamond_neighbourhood = voronoi.Neighbourhood(diamond_neighbours)

simple_voronoi_rule = "rule : {${current_cell}} 100 { (0,0)=0 and ${current_cell}!=0 and ${current_cell}!=? }"
simple_voronoi_rule_builder = voronoi.RulesetBuilder(simple_voronoi_rule)

diamond_voronoi_model = voronoi.VoronoiModel("diamondVoronoi", diamond_neighbourhood,\
                                             devs_initiall_cell_values, simple_voronoi_rule_builder).build()

In [6]:
wrapper = CDPPWrapper(diamond_voronoi_model, '00:01:00:00')
# for debug reasons, print CD++ executed command
print("" + " ".join(wrapper.getArguments()))

try:
    wrapper.run()
    drawlogNPFilename = wrapper.drawlog('00:00:00:100')    

# Simulator failure
except cdppExceptions.SimulationExecutedButFailedException:
    print(wrapper.getSimulationOutput())

# Drawlog parsing failure
except DrawlogFailedException as e:
    print(e.stderr.decode('ascii'))

cd++ -m/tmp/1543791495433.ma -o/tmp/1543791495433.out -t00:01:00:00 -l/tmp/1543791495433.log


Have to extract all this drawing logic in some kind of module, of Juputer extension. Its a lot of boilerplate.

In summary, what it does is:

1. Load drawlog-generated numpy files, with the drawn simulation
2. Conglomerate all loaded matrixes in a single one

In [77]:
%%capture

loadedMatrix = np.load(drawlogNPFilename)

composedMatrix = np.zeros([devs_initiall_cell_values.values.shape[0], devs_initiall_cell_values.values.shape[1], len(loadedMatrix.files)], dtype=np.double)

for index, npFileName in enumerate(loadedMatrix.files):
    composedMatrix[..., index] = loadedMatrix[ npFileName ].reshape(devs_initiall_cell_values.values.shape)
    
initial_values = composedMatrix[..., 0]
cells_cmap = plt.cm.Pastel1

# Set nan's to be displayed as red dots
cells_cmap.set_bad((1,0,0,1))

def do_compose_image(images, index):
    composed_image = images[..., index]
    composed_image[initial_values != 0] = np.nan
    return composed_image

index = 0
fig, ax = plt.subplots(figsize=(15,10))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plottedImage = ax.imshow(composedMatrix[..., index], cmap=cells_cmap)
stepsCount = composedMatrix.shape[2]

def updateImage(*args):
    global index
    # reuse index 0 to be shown inside drawn animation
    plottedImage.set_array(do_compose_image(composedMatrix, index))
    index += 1
    index = index % stepsCount
    return plottedImage,

In [78]:
simulationAnimation = animation.FuncAnimation(fig, updateImage, interval= 100, frames=stepsCount, repeat=False)
HTML(simulationAnimation.to_html5_video())

Iterate over several neighbourhoods, and generate animation for each, to compare them side by side.

Maybe, a neighbourhood object can be defined, and passed as parameter to the model.